<a href="https://colab.research.google.com/github/Khuliso877/Week-6-AI/blob/main/AI_driven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ======================================
# 0. SETUP
# ======================================
!pip install xgboost pandas scikit-learn matplotlib --quiet

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

np.random.seed(42)

# For nicer DataFrame display
pd.set_option('display.max_columns', 50)


# ======================================
# 1. SIMULATE SENSOR DATA
#    - Multiple plots
#    - Daily readings for one growing season
# ======================================

# CONFIG
N_PLOTS = 20            # number of different fields/plots
N_DAYS = 120            # length of growing season (days)
START_DATE = "2024-10-01"

dates = pd.date_range(start=START_DATE, periods=N_DAYS, freq='D')

# We'll simulate sensors:
# - soil_moisture_10cm
# - soil_moisture_30cm
# - soil_temp
# - air_temp
# - air_humidity
# - rainfall
# - solar_radiation
# - ndvi (simulated vegetation index)

data_rows = []

for plot_id in range(1, N_PLOTS + 1):
    # Plot-specific characteristics (static)
    soil_quality = np.random.uniform(0.6, 1.0)        # [0,1] higher = better
    irrigation_efficiency = np.random.uniform(0.5, 1.0)
    base_yield_potential = np.random.uniform(5.0, 10.0)  # tons per hectare (just for generating labels)

    # Time-series simulation
    soil_moisture_10 = np.clip(np.random.normal(loc=0.30, scale=0.05, size=N_DAYS), 0.1, 0.5)
    soil_moisture_30 = np.clip(np.random.normal(loc=0.32, scale=0.05, size=N_DAYS), 0.15, 0.55)

    # Air temperature follows a seasonal pattern + noise
    base_temp = 20 + 5 * np.sin(np.linspace(0, 2*np.pi, N_DAYS))
    air_temp = base_temp + np.random.normal(0, 1.5, N_DAYS)

    # Soil temperature slightly lagged + damped from air temp
    soil_temp = air_temp - np.random.normal(2, 0.5, N_DAYS)

    # Humidity inversely correlated a bit with temp
    air_humidity = np.clip(70 - (air_temp - 20) * 2 + np.random.normal(0, 5, N_DAYS), 30, 100)

    # Random rainfall events (mm)
    rainfall = np.random.choice([0, 0, 0, 2, 5, 10, 20], size=N_DAYS, p=[0.5,0.15,0.1,0.1,0.08,0.05,0.02])

    # Solar radiation (MJ/m²), lower on rainy/cloudy days
    solar_radiation = np.clip(np.random.normal(18, 4, N_DAYS) - rainfall * 0.2, 5, 30)

    # NDVI building up over season, peaking mid-late, then slightly decreasing
    growth_curve = 0.2 + 0.6 * np.sin(np.linspace(0, np.pi, N_DAYS))
    ndvi = np.clip(growth_curve + np.random.normal(0, 0.03, N_DAYS), 0.1, 0.9)

    for i, d in enumerate(dates):
        data_rows.append({
            "plot_id": plot_id,
            "date": d,
            "soil_moisture_10cm": soil_moisture_10[i],
            "soil_moisture_30cm": soil_moisture_30[i],
            "soil_temp": soil_temp[i],
            "air_temp": air_temp[i],
            "air_humidity": air_humidity[i],
            "rainfall": rainfall[i],
            "solar_radiation": solar_radiation[i],
            "ndvi": ndvi[i],
            "soil_quality": soil_quality,
            "irrigation_efficiency": irrigation_efficiency,
            "base_yield_potential": base_yield_potential
        })

df = pd.DataFrame(data_rows)
print("Raw simulated time-series data:")
display(df.head())


# ======================================
# 2. FEATURE ENGINEERING
#    - Aggregate daily data into season-level features
#    - These features will be used to predict yield per plot
# ======================================

# Helper: aggregate per plot over entire season
agg_funcs = {
    "soil_moisture_10cm": ["mean", "min", "max"],
    "soil_moisture_30cm": ["mean", "min", "max"],
    "soil_temp": ["mean"],
    "air_temp": ["mean", "max"],
    "air_humidity": ["mean"],
    "rainfall": ["sum"],
    "solar_radiation": ["mean"],
    "ndvi": ["mean", "max"],
    "soil_quality": ["mean"],
    "irrigation_efficiency": ["mean"],
    "base_yield_potential": ["mean"],
}

season_features = df.groupby("plot_id").agg(agg_funcs)
season_features.columns = ["_".join(col).strip() for col in season_features.columns.values]
season_features.reset_index(inplace=True)

print("\nSeason-level engineered features (one row per plot):")
display(season_features.head())


# ======================================
# 3. GENERATE SYNTHETIC YIELD LABELS
#    (Hidden "true" function + noise)
# ======================================

# We'll define a "true" yield function that depends on:
# - base_yield_potential_mean
# - average soil moisture (10 & 30 cm)
# - cumulative rainfall
# - mean NDVI
# - soil_quality_mean
# - irrigation_efficiency_mean
# - penalty for extreme temperatures

f = season_features  # alias

# normalize some features to a rough scale
soil_moisture_score = 0.5 * (f["soil_moisture_10cm_mean"] + f["soil_moisture_30cm_mean"])
rainfall_score = np.clip(f["rainfall_sum"] / 300.0, 0, 1.5)  # 300mm as some reference
ndvi_score = f["ndvi_mean"]
soil_quality = f["soil_quality_mean"]
irrigation_eff = f["irrigation_efficiency_mean"]
base_potential = f["base_yield_potential_mean"]

# Penalty for heat stress (higher mean temp reduces yield)
temp_penalty = np.clip((f["air_temp_mean"] - 22) * 0.1, -1, 1)

# "True" yield model (tons/ha)
true_yield = (
    base_potential
    * (0.4 + 0.3 * soil_moisture_score + 0.2 * rainfall_score + 0.4 * ndvi_score)
    * (0.6 + 0.4 * soil_quality)
    * (0.5 + 0.5 * irrigation_eff)
    * (1.0 - temp_penalty)
)

# Add noise
noise = np.random.normal(0, 1.0, size=len(true_yield))
yield_tpha = np.clip(true_yield + noise, 3, 15)  # restrict to realistic-ish range

season_features["yield_tons_per_ha"] = yield_tpha

print("\nSeason-level features with synthetic yield labels:")
display(season_features.head())


# ======================================
# 4. TRAIN/TEST SPLIT
# ======================================

feature_cols = [c for c in season_features.columns if c not in ["plot_id", "yield_tons_per_ha"]]
X = season_features[feature_cols]
y = season_features["yield_tons_per_ha"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")


# ======================================
# 5. TRAIN XGBoost REGRESSOR
# ======================================

model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="reg:squarederror",
    random_state=42
)

model.fit(X_train, y_train)

# ======================================
# 6. EVALUATE MODEL
# ======================================

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print("\n=== Evaluation Metrics ===")
print("MAE (tons/ha): ", round(mae, 3))
print("RMSE (tons/ha):", round(rmse, 3))
print("R^2:           ", round(r2, 3))


# Scatter plot: predicted vs actual
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--')
plt.xlabel("Actual Yield (tons/ha)")
plt.ylabel("Predicted Yield (tons/ha)")
plt.title("XGBoost - Predicted vs Actual Yield")
plt.grid(True)
plt.show()


# ======================================
# 7. FEATURE IMPORTANCE
# ======================================

importances = model.feature_importances_
feat_imp = pd.DataFrame({
    "feature": feature_cols,
    "importance": importances
}).sort_values("importance", ascending=False)

print("\nTop 10 most important features:")
display(feat_imp.head(10))

plt.figure(figsize=(8,6))
plt.barh(feat_imp["feature"].head(10)[::-1],
         feat_imp["importance"].head(10)[::-1])
plt.xlabel("Importance")
plt.title("Top 10 Feature Importances (XGBoost)")
plt.tight_layout()
plt.show()


# ======================================
# 8. SIMPLE "INFERENCE PIPELINE" EXAMPLE
#    - Pretend a new season's sensor data is aggregated
#    - Show a prediction
# ======================================

# Take one existing row and slightly perturb it to simulate a new season
new_sample = X_test.iloc[[0]].copy()
print("\nExample input feature vector for a new plot/season:")
display(new_sample)

pred_yield = model.predict(new_sample)[0]
print(f"Predicted yield for this new plot/season: {pred_yield:.2f} tons/ha")


Raw simulated time-series data:


,plot_id,date,soil_moisture_10cm,soil_moisture_30cm,soil_temp,air_temp,air_humidity,rainfall,solar_radiation,ndvi,soil_quality,irrigation_efficiency,base_yield_potential
0,1,2024-10-01,0.244406,0.279909,16.733046,19.333252,72.424610,5,12.033995,0.167883,0.749816,0.975357,8.65997
1,1,2024-10-02,0.315945,0.389201,14.852134,17.760178,79.032464,0,14.734778,0.236317,0.749816,0.975357,8.65997
2,1,2024-10-03,0.313952,0.390260,17.424755,19.369289,76.538076,0,23.263066,0.226694,0.749816,0.975357,8.65997
3,1,2024-10-04,0.350526,0.389616,20.792488,23.280643,66.706885,0,10.987572,0.222703,0.749816,0.975357,8.65997
4,1,2024-10-05,0.270956,0.275968,19.273219,20.546794,64.536542,10,19.025406,0.332534,0.749816,0.975357,8.65997



Season-level engineered features (one row per plot):


,plot_id,soil_moisture_10cm_mean,soil_moisture_10cm_min,soil_moisture_10cm_max,soil_moisture_30cm_mean,soil_moisture_30cm_min,soil_moisture_30cm_max,soil_temp_mean,air_temp_mean,air_temp_max,air_humidity_mean,rainfall_sum,solar_radiation_mean,ndvi_mean,ndvi_max,soil_quality_mean,irrigation_efficiency_mean,base_yield_potential_mean
0,1,0.296538,0.167241,0.468615,0.331976,0.230575,0.456721,18.231064,20.214272,28.518076,69.659617,160,17.326094,0.578348,0.845898,0.749816,0.975357,8.659970
1,2,0.304673,0.157573,0.407114,0.325126,0.175187,0.441988,18.100389,20.109209,28.507678,70.508965,211,18.347215,0.575965,0.839768,0.747411,0.932179,7.366050
2,3,0.299056,0.179013,0.435797,0.317503,0.178374,0.429374,17.743621,19.769375,27.271561,70.280571,133,18.720822,0.575674,0.841025,0.995214,0.727001,8.441371
3,4,0.296132,0.150443,0.496312,0.316204,0.178392,0.440584,17.785052,19.842788,27.430496,70.608778,188,17.831762,0.578242,0.885570,0.714283,0.963562,9.849633
4,5,0.300889,0.155005,0.420711,0.322084,0.190579,0.445082,18.028801,19.978847,27.992140,69.818869,168,17.682832,0.580902,0.870333,0.665715,0.891581,5.612489



Season-level features with synthetic yield labels:


,plot_id,soil_moisture_10cm_mean,soil_moisture_10cm_min,soil_moisture_10cm_max,soil_moisture_30cm_mean,soil_moisture_30cm_min,soil_moisture_30cm_max,soil_temp_mean,air_temp_mean,air_temp_max,air_humidity_mean,rainfall_sum,solar_radiation_mean,ndvi_mean,ndvi_max,soil_quality_mean,irrigation_efficiency_mean,base_yield_potential_mean,yield_tons_per_ha
0,1,0.296538,0.167241,0.468615,0.331976,0.230575,0.456721,18.231064,20.214272,28.518076,69.659617,160,17.326094,0.578348,0.845898,0.749816,0.975357,8.659970,7.878347
1,2,0.304673,0.157573,0.407114,0.325126,0.175187,0.441988,18.100389,20.109209,28.507678,70.508965,211,18.347215,0.575965,0.839768,0.747411,0.932179,7.366050,6.923435
2,3,0.299056,0.179013,0.435797,0.317503,0.178374,0.429374,17.743621,19.769375,27.271561,70.280571,133,18.720822,0.575674,0.841025,0.995214,0.727001,8.441371,6.039386
3,4,0.296132,0.150443,0.496312,0.316204,0.178392,0.440584,17.785052,19.842788,27.430496,70.608778,188,17.831762,0.578242,0.885570,0.714283,0.963562,9.849633,7.790854
4,5,0.300889,0.155005,0.420711,0.322084,0.190579,0.445082,18.028801,19.978847,27.992140,69.818869,168,17.682832,0.580902,0.870333,0.665715,0.891581,5.612489,5.572543


Train size: 14, Test size: 6


TypeError: got an unexpected keyword argument 'squared'